In [ ]:
#SimPy
    #Basic structure of a SimPy model:
        #
    #Non-linear one sequential activities - this requires more information to be input as we have to choose which area an element passes
    #make a decision branch with an if statement. For simulation, use a uniform distribution for a probability (between 0 and 1) and this can randomly generate a probabltility of going down a route. Then compare to your defining percentages (0.2 and 0.8 in the example) and that will choose the route to which the element proceeds.
    #Referring to a variable not inside a function, need to use the global keyword to pull variables outside in the global environment. Not the best approach - shows need for OOP.
    #If we want multuple generators, then define generators at top. Don't forget to start multiple processes if you have them!
    #If you want to get the results, since these are stochastic (random) models, then we need to run multiple times. To do so, put in a for loop and set the number of runs. Then you can save out the results into a global object or save to a file (.csv) before the next run starts on a new line. You can then import the results and do some statistics on this to get a better idea (high power) of real world results.
    #We are currently assuming our entities are empty when we first start the simulation. We can do this in some instances (GP with no patients at opening and no patients at closing). If modelling 24 hour systems, we need to have entitites in there to start. We can use a warm up period to get entitites in the system. Can approach warm up period in different ways depending on system being modeled. Set teh warm up period then do if statement before results are collected on each activity. 
    #Containers - type of class that allows you to pool a resource and draw from this while the simulation runs.

In [3]:
# SimPy pt. 1
## Exercise 2
import simpy
import random
import matplotlib.pyplot as plt
from statistics import mean
import csv
import pandas as pd

# Patient arrival generator function
def patient_generator(env, gp_inter, mean_consult, mean_register, mean_book, receptionist, gp_doctor):
    p_id = 0
    while True:
        p = patient_generator(env, mean_register, mean_consult, mean_book, receptionist, gp_doctor, p_id)
        env.process(p)
        t = random.expovariate(1/gp_inter)
        yield env.timeout(t)
        p_id == 1
# Calls generator function
def call_generator(env, call_inter, mean_call, receptionist):
    c_id = 0
    while True:
        c = call_generator(env, call_inter, mean_call, receptionist)
        env.process(c)
        tc = random.expovariate(1/call_inter)
        yield env.timeout(tc)
        c_id == 1

    # Call activity generator
    def activity_generator_call(env, mean_call, receptionist, c_id):
        global list_of_queueing_times_call
        time_entered_queue_for_call = env.now
        with receptionist.request() as req:
            yield req
            time_left_queue_for_call = env.now
            time_in_queue_for_call = (time_left_queue_for_call - time_entered_queue_for_call)
            print(f'Patient {p_id} queued for call for',
                  f'{time_in_queue_for_call:.1f} minutes.')
            list_of_queueing_times_call.append(time_in_queue_for_call)
            sampled_call_time = random.expovariate(1/mean_call_answer)
            yield env.timeout(sampled_call_time)
    # Visit activity generator
    def activity_generator_doc(env, mean_resgister, mean_consult, mean_book, receptionist, gp_doctor, p_id):
        global list_of_queueing_times_register
        global warm_up_period
        with receptionist.request() as req:
            time_entered_queue_for_registration = env.now
            yield req
            time_left_queue_for_registration = env.now
            time_in_queue_for_registration = (time_left_queue_for_registration - time_entered_queue_for_registration)
            print(f'Patient {p_id} queued for registration for',
             f'{time_in_queue_for_registration:.1f} minutes.')
            list_of_queueing_times_register.append(time_in_queue_for_book)
            sample_registration_time = random.expovariate(1/mean_register)
            yield env.timeout(sample_registration_time)
        time_entered_queue_for_doctor = env.now
        
        with doctor.request() as req:
            yield req
            time_left_queue_for_doctor = env.now
            time_in_queue_for_doctor = (time_left_queue_for_doctor - time_entered_queue_for_doctor)
            print(f'Patient {p_id} queued for doctor for', f'{time_in_queue_for_doctor:.1f} minutes.')
            list_of_queueing_times_consult.append(time_in_queue_for_doctor)
            sampled_doctor_time = random.expovariate(1/mean_consult)
            yield env.timeout(sampled_doctor_time)
       
        decide_book_branch = random.uniform(0,1)
        
        if decide_book_branch <= 0.25:
            time_entered_queue_for_book = env.now
            with book.request() as req:
                yield req
                time_left_queue_for_book = env.now
                time_in_queue_for_book = (time_left_queue_for_book - time_entered_queue_for_book)
                print(f'Patient {p_id} queued for booking for', 
                      f'{time_in_queue_for_book:.1f} minutes.')
                list_of_queueing_times_book.append(time_in_queue_for_book)
                sampled_book_time = random.expovariate(1/mean_book)
                yield env.timeout(sampled_book_time)
                
        total_time = env.now - time_entered_queue_for_registration
        
        list_of_total_time.append(total_time)

number_of_runs = 100
results_collection = 480
warm_up = 180

with open('sim_results.csv', 'w') as f:
    writer = csv.writer(f, delimiter = ',')
    writer.writerow(['Run', 'Length of stay'])
        
# Create for loop to run a certain number of times
for run in range(number_of_runs):

    # Set up simulation environment
    env = simpy.Environment()

    # Resources
    receptionist = simpy.Resource(env, capacity = 1)
    gp_doctor = simpy.Resource(env, capacity = 2)

    # Set up parameter values
    gp_inter = 3
    mean_register = 2
    mean_consult = 8
    mean_book = 4
    mean_call = 4
    call_inter = 10

    # Set up list for queueing times
    list_of_queueing_times_register = []
    list_of_queueing_times_consult = []
    list_of_queueing_times_book = []
    list_of_total_time = []
    list_of_queueing_times_call = []

    # Start the arrivals generator
    env.process(patient_generator(env, gp_inter, mean_consult, mean_register, mean_book, receptionist, gp_doctor))
    env.process(call_generator(env, call_inter, mean_call, receptionist))

    # Run the simulation
    env.run(until = (results_collection + warm_up))

    # Calculate means and total and print
    mean_queue_time_register = mean(list_of_queueing_times_register)
    print(f'Mean queueing time for registration is {mean_queue_time_register:.2f} minutes.')
    mean_queue_time_consult = mean(list_of_queueing_times_consult)
    print(f'Mean queueing time for consult is {mean_queue_time_consult:.2f} minutes.')
    mean_queue_time_book = mean(list_of_queueing_times_book)
    print(f'Mean queueing time for booking is {mean_queue_time_book:.2f} minutes.')
    mean_queue_time_call = mean(list_of_queueing_times_call)
    print(f'Mean queueing time for calls is {mean_queue_time_call:.2f} minutes.')
    mean_total_time = mean(list_of_total_time)
    print(f'Mean total queueing time is {mean_total_time:.2f} minutes.')

    # Set up list to write to csv file
    list_to_write = [run, mean_queue_time_register, mean_queue_time_consult, mean_queue_time_book, mean_queue_time_call, mean_total_time]
      
# create df of the results
df = pd.read_csv('sim_results.csv')
      ## Need to finish adding in the calls and the mean times for these.

In [ ]:
# SimPy pt. 1
## Exercise 1
import simpy
import random
import matplotlib.pyplot as plt
from statistics import mean

# Arrivals generator function
def patient_generator(env, gp_inter, mean_consult, mean_register, mean_book, receptionist, gp_doctor):
    p_id = 0
    
    # Arrivals generator
    while True:
        p = patient_generator(env, mean_register, mean_consult, mean_book, receptionist, gp_doctor, p_id)
        env.process(p)
        t = random.expovariate(1/gp_inter)
        yield env.timeout(t)
        p_id == 1
        
    # Activity generator function
    def activity_generator(env, mean_resgister, mean_consult, mean_book, receptionist, gp_doctor, p_id):
        with receptionist.request() as req:
            time_entered_queue_for_registration = env.now
            yield req
            time_left_queue_for_registration = env.now
            time_in_queue_for_registration = (time_left_queue_for_registration - time_entered_queue_for_registration)
            print(f'Patient {p_id} queued for registration for',
             f'{time_in_queue_for_registration:.1f} minutes.')
            list_of_queueing_times_register.append(time_in_queue_for_book)
            sample_registration_time = random.expovariate(1/mean_register)
            yield env.timeout(sample_registration_time)
        time_entered_queue_for_doctor = env.now
        
        with doctor.request() as req:
            yield req
            time_left_queue_for_doctor = env.now
            time_in_queue_for_doctor = (time_left_queue_for_doctor - time_entered_queue_for_doctor)
            print(f'Patient {p_id} queued for doctor for', f'{time_in_queue_for_doctor:.1f} minutes.')
            list_of_queueing_times_consult.append(time_in_queue_for_doctor)
            sampled_doctor_time = random.expovariate(1/mean_consult)
            yield env.timeout(sampled_doctor_time)
            
        decide_book_branch = random.uniform(0,1)
        
        if decide_book_branch <= 0.25:
            time_entered_queue_for_book = env.now
            with book.request() as req:
                yield req
                time_left_queue_for_doctor = env.now
                time_in_queue_for_book = (time_left_queue_for_doctor - time_entered_queue_for_book)
                print(f'Patient {p_id} queued for booking for', f'{time_in_queue_for_book:.1f} minutes.')
                list_of_queueing_times_book.append(time_in_queue_for_book)
                sampled_book_time = random.expovariate(1/mean_book)
                yield env.timeout(sampled_book_time)
                
        total_time = env.now - time_entered_queue_for_registration
        
        list_of_total_time.append(total_time)
        
# Set up simulation environment
env = simpy.Environment()

# Resources
receptionist = simpy.Resource(env, capacity = 1)
gp_doctor = simpy.Resource(env, capacity = 2)

# Set up parameter values
gp_inter = 3
mean_register = 2
mean_consult = 8
mean_book = 4

# Set up list for queueing times
list_of_queueing_times_register = []
list_of_queueing_times_consult = []
list_of_queueing_times_book = []
list_of_total_time = []

# Start the arrivals generator
env.process(patient_generator(env, gp_inter, mean_consult, mean_register, mean_book, receptionist, gp_doctor))

# Run the simulation
env.run(until = 480)

# Calculate means and total and print
mean_queue_time_register = mean(list_of_queueing_times_register)
print(f'Mean queueing time for registration is {mean_queue_time_register:.2f} minutes.')
mean_queue_time_consult = mean(list_of_queueing_times_consult)
print(f'Mean queueing time for consult is {mean_queue_time_consult:.2f} minutes.')
mean_queue_time_book = mean(list_of_queueing_times_book)
print(f'Mean queueing time for booking is {mean_queue_time_book:.2f} minutes.')
mean_total_time = mean(list_of_total_time)
print(f'Mean total queueing time is {mean_queue_total_time:.2f} minutes.')

# Plot and display
      
fig, ax = plt.subplots()
      
counts = [mean_queue_time_register, mean_queue_time_consult, mean_test_book]

names = ["Registration", "Consult", "Booking"]

ax.bar(names, counts)

plt.show()